In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm
from datetime import datetime  

In [2]:
# df = pd.read_csv("../data/hackathon_data.csv")
# df.head()

In [3]:
# def get_uniques_stats(df):
#     unique_stats = {}
#     for col in df.columns:
#         unique_stats[col] = len(df[col].unique())
#     print(unique_stats)
#     print()

In [4]:
# get_uniques_stats(df)

### Hierarchy 
1. 7 cites

In [5]:
# cities = df.city_id.unique()
# for city in cities:
#     print(city)
#     get_uniques_stats(df.loc[df.city_id ==city])

In [6]:
# city = 1120784
# df = df.loc[df.city_id == city]
# # pd.DataFrame(df.society_id.unique(),columns=["city_id"]).to_csv("societies_of_selected_society.csv",index=None)

In [3]:
df = pd.read_csv("City_Subset.csv")
df.head()

,customer_id,manufacturer_id,society_id,city_id,route_id,store_id,order_id,order_date,category_id,subcategory_id,...,order_placed_month,order_placed_day_of_week,order_placed_hour,order_day,order_week,order_month,order_day_of_week,lat,long,area
0,20049456,1148224,1267056,1120784,1138032,1120112,734536432,2019-01-27 00:00:00,1122464,1124816,...,1,5,12,27,4,1,6,18.485728,73.802289,0
1,20164592,1122016,1256416,1120784,1137248,1120112,734763008,2019-01-27 00:00:00,1122576,1125152,...,1,5,20,27,4,1,6,18.469226,73.803260,0
2,20049456,1155504,1267056,1120784,1138032,1120112,734536432,2019-01-27 00:00:00,1122800,1126496,...,1,5,13,27,4,1,6,18.485728,73.802289,0
3,20049456,1127728,1267056,1120784,1138032,1120112,734536432,2019-01-27 00:00:00,1122800,1126160,...,1,5,13,27,4,1,6,18.485728,73.802289,0
4,20164592,1150128,1256416,1120784,1137248,1120112,734763008,2019-01-27 00:00:00,1123472,1130640,...,1,5,20,27,4,1,6,18.469226,73.803260,0


In [ ]:
# df = pd.read_csv("City_Subset.csv")
df.order_date = pd.to_datetime(df.order_date,format='%Y-%m-%d') 
df.product_addedtobasket_on = pd.to_datetime(df.product_addedtobasket_on,format='%Y-%m-%d %H:%M') 


df["order_placed_date"] = df.product_addedtobasket_on.apply(lambda x: x.date())
df["order_placed_day"] = df.product_addedtobasket_on.apply(lambda x: x.day)
df["order_placed_month"] = df.product_addedtobasket_on.apply(lambda x: x.month)
df["order_placed_day_of_week"] = df.product_addedtobasket_on.apply(lambda x: x.dayofweek)
df["order_placed_hour"] = df.product_addedtobasket_on.apply(lambda x: x.hour)
df["order_day"] = df.order_date.apply(lambda x: x.day)
df["order_week"] = df.order_date.apply(lambda x: x.isocalendar()[1])
df["order_month"] = df.order_date.apply(lambda x: x.month)
df["order_day_of_week"] = df.order_date.apply(lambda x: x.dayofweek)

# # 

In [ ]:
df.head()

In [ ]:
df.to_csv("City_Subset_with_TimeSeries.csv",index=None)

#### Same Society can fall into different routes

1. Society_ID and Route Id 
Hierarchy:
1. City -> Route_ID -> Society_ID
2. City -> Store_ID ->Route_ID -> Society_ID

Need to understand the following
1. Subscription
2. manufacturer_id

## User Specific Analysis

In [8]:
df = pd.read_csv("../data/City_Subset_with_TimeSeries.csv")

df.order_date = pd.to_datetime(df.order_date,format='%Y-%m-%d') 
df.product_addedtobasket_on = pd.to_datetime(df.product_addedtobasket_on,format='%Y-%m-%d %H:%M') 

In [9]:
society_lat_long = pd.read_csv("../data/societies_latlong.csv")
# society_lat_long.head()
society_lat_long = society_lat_long.set_index("society_id")
df["lat"] = df.society_id.apply(lambda x: society_lat_long["lat"][x])
df["long"] = df.society_id.apply(lambda x: society_lat_long["long"][x])
df["area"] = df.society_id.apply(lambda x: society_lat_long["area"][x])

In [10]:
df["lat"] = df.society_id.apply(lambda x: society_lat_long["lat"][x])
df["long"] = df.society_id.apply(lambda x: society_lat_long["long"][x])
df["area"] = df.society_id.apply(lambda x: society_lat_long["area"][x])
# df

In [11]:
def volume_quantile(splits, var, df, ID):    
    df = df.sort_values([var])
    df['CUM_VAR'] = df[var].cumsum()/df[var].sum()
    
    #restamp values >1
    mask = df['CUM_VAR'] > 1
    column_name = 'CUM_VAR'
    df.loc[mask, column_name] = 1
    
    #create buckets
    df['quintile'] = (df['CUM_VAR']*splits)
    df['quintile'] = df['quintile'].apply(np.ceil)
    
    #stat summary
#     results.groupby(['decile'])['mrkt_trx_vol'].sum()
    print(df.groupby('quintile').agg({ID:'count', var: ['sum','max','min']}).reset_index())
    
    return df


In [12]:
import progressbar
pbar = progressbar.ProgressBar()
number_of_weeks = max(df.order_week)
customer_data = []

customers = df.customer_id.unique()
df.order_date = pd.to_datetime(df.order_date,format='%Y-%m-%d') 
# df.product_addedtobasket_on = pd.to_datetime(df.product_addedtobasket_on,format='%Y-%m-%d %H:%M') 
date_range = int(str(max(df.order_date) - min(df.order_date)).split(" ")[0])

for customer in pbar(customers):
    customer_df = df.loc[df.customer_id==customer]
    
    customer_df=customer_df.reset_index()
    customer_row = {}
    customer_row["customer_id"] = customer
    customer_row["society_id"] = customer_df.society_id[0]
    customer_row["mean_order_cost"] = customer_df.groupby(by="order_date")["total_cost"].sum().mean()
    customer_row["mean_weekly_cost"] = customer_df.groupby(by="order_week")["total_cost"].sum().sum()/number_of_weeks
    customer_row["mean_daily_cost"] = (customer_df.groupby(by="order_week")["total_cost"].sum()).sum()/date_range
    customer_data.append(customer_row)
customer_data = pd.DataFrame(customer_data)
customer_data = volume_quantile(5,"mean_weekly_cost",customer_data,"quintile")
# customer_data


100% |########################################################################|


  quintile       mean_weekly_cost                         
           count              sum          max         min
0      1.0   282     79997.115880   361.845484    3.941613
1      2.0   195     79981.840025   455.260161  361.875161
2      3.0   160     80200.889497   558.673871  456.325649
3      4.0   128     80350.698929   708.007668  558.966345
4      5.0    96     80453.750620  1384.566200  708.631677


In [14]:
customer_data.to_csv("../output/customer_intermediate.csv",index=None)

In [8]:
df.head()

,customer_id,manufacturer_id,society_id,city_id,route_id,store_id,order_id,order_date,category_id,subcategory_id,...,order_placed_month,order_placed_day_of_week,order_placed_hour,order_day,order_week,order_month,order_day_of_week,lat,long,area
0,20049456,1148224,1267056,1120784,1138032,1120112,734536432,2019-01-27,1122464,1124816,...,1,5,12,27,4,1,6,18.485728,73.802289,0
1,20164592,1122016,1256416,1120784,1137248,1120112,734763008,2019-01-27,1122576,1125152,...,1,5,20,27,4,1,6,18.469226,73.803260,0
2,20049456,1155504,1267056,1120784,1138032,1120112,734536432,2019-01-27,1122800,1126496,...,1,5,13,27,4,1,6,18.485728,73.802289,0
3,20049456,1127728,1267056,1120784,1138032,1120112,734536432,2019-01-27,1122800,1126160,...,1,5,13,27,4,1,6,18.485728,73.802289,0
4,20164592,1150128,1256416,1120784,1137248,1120112,734763008,2019-01-27,1123472,1130640,...,1,5,20,27,4,1,6,18.469226,73.803260,0


In [51]:
number_of_weeks = max(df.order_week)
area_data = []
pbar = progressbar.ProgressBar()
areas = df.area.unique()

for area in tqdm(areas):
    area_df = df.loc[df.area==area]
    
    area_df = area_df.reset_index()
    area_row = {}
    area_row["area"] = area
#     pod_row["lat"] = society_df.lat[0]
#     society_row["long"] = society_df.long[0]
#     society_row["customer_id"] = society_df.customer_id[0]
    area_row["mean_order_cost"] = area_df.groupby(by="order_date")["total_cost"].sum().mean()
    area_row["mean_weekly_cost"] = area_df.groupby(by="order_week")["total_cost"].sum().sum()/number_of_weeks
    area_row["mean_daily_cost"] = (area_df.groupby(by="order_week")["total_cost"].sum()).sum()/date_range
    area_data.append(area_row)
area_data = pd.DataFrame(area_data)
area_data = volume_quantile(5,"mean_weekly_cost",area_data,"quintile")
# society_data

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 40.71it/s]


  quintile       mean_weekly_cost                            
           count              sum           max           min
0      1.0     4     60601.821477  21866.231947   3158.173441
1      2.0     3     77058.884292  25713.297041  25654.110733
2      3.0     3     94239.723914  33320.868010  28688.177391
3      4.0     2     75766.233319  38950.624467  36815.608852
4      5.0     2     93317.631948  48561.629285  44756.002663


In [13]:
number_of_weeks = max(df.order_week)
society_data = []
pbar = progressbar.ProgressBar()
societies = df.society_id.unique()
date_range = int(str(max(df.order_date) - min(df.order_date)).split(" ")[0])

for society in pbar(societies):
    society_df = df.loc[df.society_id==society]
    
    society_df=society_df.reset_index()
    society_row = {}
    society_row["society_id"] = society
    society_row["area"] = society_df.area[0]
    society_row["lat"] = society_df.lat[0]
    society_row["long"] = society_df.long[0]
#     society_row["customer_id"] = society_df.customer_id[0]
    society_row["mean_order_cost"] = society_df.groupby(by="order_date")["total_cost"].sum().mean()
    society_row["mean_weekly_cost"] = society_df.groupby(by="order_week")["total_cost"].sum().sum()/number_of_weeks
    society_row["mean_daily_cost"] = (society_df.groupby(by="order_week")["total_cost"].sum()).sum()/date_range
    society_data.append(society_row)
society_data = pd.DataFrame(society_data)
society_data = volume_quantile(5,"mean_weekly_cost",society_data,"quintile")
# society_data

100% |########################################################################|


  quintile       mean_weekly_cost                          
           count              sum          max          min
0      1.0   149     79446.562446   959.314277     5.005484
1      2.0    65     79748.836754  1453.956604   960.300484
2      3.0    46     80063.817862  1993.551427  1454.786557
3      4.0    34     79205.073399  2785.329484  1995.697751
4      5.0    20     82520.004490  8597.832726  2785.484097


In [116]:
# pbar = progressbar.ProgressBar()
# pods = df.pod.unique()
# date_range = int(str(max(df.order_date) - min(df.order_date)).split(" ")[0])
# product_society = []
# from joblib import Parallel, delayed
# from tqdm import tqdm

# def get_product_data(pod):
#     pod_df = df.loc[df.pod==pod]
#     products = pod_df.product_id.unique()
#     product_data = []
#     final = {}
#     for product in products:
#         product_df = pod_df.loc[pod_df.product_id == product]
#         product_df=product_df.reset_index()
#         product_row = {}
#         product_row["product_id"] = product
#         product_row["mean_order_cost"] = product_df.groupby(by="order_date")["total_cost"].sum().mean()
#         product_row["mean_weekly_cost"] = product_df.groupby(by="order_week")["total_cost"].sum().sum()/number_of_weeks
#         product_row["mean_daily_cost"] = (product_df.groupby(by="order_week")["total_cost"].sum()).sum()/date_range
#         product_data.append(product_row)
#     product_data = pd.DataFrame(product_data)

# #     product_data = volume_quantile(5,"mean_weekly_cost",product_data,"quintile")
#     final[pod] = product_data
#     return final
# # pod_product = Parallel(n_jobs=8)(delayed(get_product_data)(pod) for pod in tqdm(pods))
# pod_product = []
# for pod in tqdm(pods):
#     pod_product.append(get_product_data(pod))
#     break
# pod_product[0]

In [117]:
# pod_product[9]

In [118]:
# society_product

In [119]:
society_data.to_csv("../output/society.csv",index=None)

In [148]:
Area_centers = [
  {
    "lat": 18.464815,
    "long": 73.796439
  },
  {
    "lat": 18.507762,
    "long": 73.798775
  },
  {
    "lat": 18.52517,
    "long": 73.779009
  },
  {
    "lat": 18.56792,
    "long": 73.770989
  },
  {
    "lat": 18.556043,
    "long": 73.812341
  },
  {
    "lat": 18.683466,
    "long": 73.731081
  },
  {
    "lat": 18.665461,
    "long": 73.808566
  },
  {
    "lat": 18.565205,
    "long": 73.911494
  },
  {
    "lat": 18.529224,
    "long": 73.860415
  },
  {
    "lat": 18.435227,
    "long": 73.889246
  },
  {
    "lat": 18.50861,
    "long": 73.934341
  },
  {
    "lat": 18.635565,
    "long": 73.843959
  },
  {
    "lat": 18.606168,
    "long": 73.874991
  },
  
  {
    "lat": 18.599301,
    "long": 73.927049
  },
  {
    "lat": 18.59913,
    "long": 73.737242
  }
]

warehouse = {
    "lat": 18.676853,
    "long": 73.897229
  }
len(Area_centers)

15

In [149]:
area_lat = []
area_long = []
for i in range(0,len(df.area.unique())):
    temp = df.loc[df.area == i]
    temp = temp.reset_index()
    area_lat.append(temp.lat[0])
    area_long.append(temp.long[0])

In [150]:
import geopy.distance

def getDistance(lat1, long1, lat2=warehouse['lat'], long2=warehouse['long']):
    return geopy.distance.distance((lat1, long1), (lat2, long2)).km
# print(len(Area_centers))
# print(len(distance_from_ware_house))

In [151]:
distance_from_ware_house = []
for i in range(0,len(df.area.unique())):
    distance_from_ware_house.append(getDistance(area_lat[i],area_long[i]))
distance_from_ware_house


[23.408587636109765,
 20.98645022630737,
 19.861086474058528,
 17.00805355415304,
 16.536194774308466,
 18.165195229996627,
 7.329556595698092,
 12.295152117974828,
 18.66078752896899,
 27.946354920052126,
 17.022022528930627,
 7.074805049948809,
 7.072476862470887,
 8.244048838148595]

In [152]:
area_priority=society_data.groupby("area")["quintile"].mean()
area_priority = area_priority.tolist()
len(distance_from_ware_house)

14

In [153]:
area_priority=society_data.groupby("area")["quintile"].mean()
pod_check = pd.DataFrame()
pod_check["area_priority"] = area_priority
pod_check["lat"] = area_lat
pod_check["long"] = area_long
# pod_check["area_id"] = pod_check.index
pod_check["distance_from_ware_house"] = distance_from_ware_house
pod_check = pod_check.sort_values(["area_priority"],ascending=False)
pod_check = pod_check.reset_index()

In [154]:
pod_check

,area,area_priority,lat,long,distance_from_ware_house
0,0,2.666667,18.485728,73.802289,23.408588
1,2,2.636364,18.526163,73.795066,19.861086
2,5,2.629630,18.681886,73.725115,18.165195
3,6,2.481481,18.670743,73.828048,7.329557
4,3,2.388889,18.572750,73.778681,17.008054
5,1,2.269231,18.513894,73.795588,20.986450
6,4,2.266667,18.546868,73.819992,16.536195
7,8,2.181818,18.512602,73.857374,18.660788
8,10,1.960000,18.525055,73.923069,17.022023
9,9,1.913043,18.425186,73.875990,27.946355


# def get_pods(col):
distance_from_pod_1

In [155]:
def opt(col,p):
    pods = []
    for index,row in pod_check.iterrows():
        if index < p:
            continue
        if row.area_priority > 2.2:
            if row[col] > 5:
                
                distance_from_pod = []
                new_pod = row
                
                for i in range(0,len(df.area.unique())):
                    distance_from_pod.append(getDistance(pod_check.lat[i],pod_check.long[i],new_pod.lat,new_pod.long))
#                 pod_check["distance_from_pod"+str(p)] = distance_from_pod
                return distance_from_pod


In [157]:
pod_check.to_csv("../output/pod_data.csv",index=None)

In [81]:
# col = ["distance_from_ware_house"]

pods = []
for index,row in pod_check.iterrows():
    if row.area_priority > 2.2:
        distance_from_pod_1 = []
        if row.distance_from_ware_house > 5:
            pods.append(row.area)
            for i in range(0,len(df.area.unique())):
                distance_from_pod_1.append(getDistance(pod_check.lat[i],pod_check.long[i],pods[],pods[]))
            break
distance_from_ware_house = []


In [82]:
distance_from_pod_1

[23.408587636109765,
 19.861086474058528,
 18.165195229996627,
 7.329556595698092,
 17.00805355415304,
 20.98645022630737,
 16.536194774308466,
 18.66078752896899,
 17.022022528930627,
 27.946354920052126,
 12.295152117974828,
 7.074805049948809,
 7.072476862470887,
 8.244048838148595]

In [56]:


# area_high_priority = []
# area_low_priority = []
# for i in range(0,len(area_priority)):
#     if area_priority[i]>2.5:
#         area_high_priority.append(i)
#     else:
#         area_low_priority.append(i)
# area_high_priority

[0, 2, 5]

In [ ]:
distance_from_ware_house = []
for i in range(0,len(df.area.unique())):
    distance_from_ware_house.append(getDistance(area_lat[i],area_long[i]))
distance_from_ware_house

In [163]:
area_data.to_csv("../output/area_intermediate.csv",index=None)